#  Предсказания закрытия стартапов

**Цель проекта**



## Загрузка данных

In [1]:
# #Устанавливаем необходимые компоненты
# !pip install phik -q
# !pip install --upgrade scikit-learn -q
# !pip install shap -q

In [2]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats as stb
import math
import os
from IPython.display import display

#import phik
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, roc_auc_score
#import shap

RANDOM_STATE = 42
TEST_SIZE = 0.25

pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
sns.set_theme(style='darkgrid')
pd.set_option('display.float_format', '{:,.3f}'.format)

In [3]:
# Загружаем данные из файлов в датафремы

pth1, pth2, pth3 = (r"datasets\kaggle_startups_train_28062024.csv",
                    r"datasets\kaggle_startups_test_28062024.csv",
                    r"datasets\kaggle_startups_sample_submit_28062024.csv")

if os.path.exists(pth1) and os.path.exists(pth2) and os.path.exists(pth3):
    startups_train = pd.read_csv(pth1, sep=',')
    startups_test = pd.read_csv(pth2, sep=',')
    target_test = pd.read_csv(pth3, sep=',')
else:
    print('Проверьте правильность написания пути к файлам или их доступность по указанному адресу')

In [4]:
# Составляем список датафреймов и их имен
dfs = [startups_train, startups_test, target_test]
name_dfs = ['startups_train', 'startups_test', 'target_test']

# Функция для отображения информации по df, первые строки, типы данных, размер
def data_view(df):
    display(df.head())
    print('\n\033[1mИнформация по составу датафрейма:\033[0m\n')
    df.info()
    print('\n\n\n')

In [5]:
# Цикл для просмотра информации по данным датафремов
def df_view(dfs, name_dfs):
    i = 0
    for df in dfs:
        print(f'\033[1mДатафрейм {name_dfs[i]}:\033[0m')
        data_view(df)
        i += 1

In [6]:
df_view(dfs, name_dfs)

Датафрейм startups_train:


,name,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at,closed_at
0,Lunchgate,Online Reservations|Restaurants,"828,626.000",operating,CHE,25,Zurich,Zürich,2,2009-10-17,2011-05-01,2014-12-01,NaN
1,EarLens,Manufacturing|Medical|Medical Devices,"42,935,019.000",operating,USA,CA,SF Bay Area,Redwood City,4,2005-01-01,2010-05-04,2014-02-25,NaN
2,Reviva Pharmaceuticals,Biotechnology,"35,456,381.000",operating,USA,CA,SF Bay Area,San Jose,3,2006-01-01,2012-08-20,2014-07-02,NaN
3,Sancilio and Company,Health Care,"22,250,000.000",operating,NaN,NaN,NaN,NaN,3,2004-01-01,2011-09-01,2014-07-18,NaN
4,WireTough Cylinders,Manufacturing,NaN,operating,USA,VA,VA - Other,Bristol,1,2010-05-12,2012-02-01,2012-02-01,NaN



Информация по составу датафрейма:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52516 entries, 0 to 52515
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               52515 non-null  object 
 1   category_list      50051 non-null  object 
 2   funding_total_usd  42447 non-null  float64
 3   status             52516 non-null  object 
 4   country_code       47014 non-null  object 
 5   state_code         45753 non-null  object 
 6   region             46157 non-null  object 
 7   city               46157 non-null  object 
 8   funding_rounds     52516 non-null  int64  
 9   founded_at         52516 non-null  object 
 10  first_funding_at   52516 non-null  object 
 11  last_funding_at    52516 non-null  object 
 12  closed_at          4917 non-null   object 
dtypes: float64(1), int64(1), object(11)
memory usage: 5.2+ MB




Датафрейм startups_test:


,name,category_list,funding_total_usd,country_code,state_code,region,city,funding_rounds,first_funding_at,last_funding_at,lifetime
0,Crystalsol,Clean Technology,"2,819,200.000",NIC,17,NaN,NaN,1,2009-07-01,2009-07-01,3501
1,JBI Fish & Wings,Hospitality,NaN,USA,TN,TN - Other,Humboldt,1,2010-07-28,2010-07-28,2717
2,COINPLUS,Finance,"428,257.000",LUX,3,Esch-sur-alzette,Esch-sur-alzette,2,2014-05-15,2014-09-18,1295
3,Imagine Communications,Software|Video|Video Streaming,"34,700,000.000",USA,CA,San Diego,San Diego,4,2005-01-01,2010-04-20,4748
4,DNA13,Software,"4,530,000.000",CAN,ON,Ottawa,Ottawa,1,2007-05-08,2007-05-08,6209



Информация по составу датафрейма:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13125 entries, 0 to 13124
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               13125 non-null  object 
 1   category_list      12534 non-null  object 
 2   funding_total_usd  10547 non-null  float64
 3   country_code       11743 non-null  object 
 4   state_code         11430 non-null  object 
 5   region             11536 non-null  object 
 6   city               11538 non-null  object 
 7   funding_rounds     13125 non-null  int64  
 8   first_funding_at   13125 non-null  object 
 9   last_funding_at    13125 non-null  object 
 10  lifetime           13125 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 1.1+ MB




Датафрейм target_test:


,name,status
0,Crystalsol,closed
1,JBI Fish & Wings,operating
2,COINPLUS,closed
3,Imagine Communications,closed
4,DNA13,operating



Информация по составу датафрейма:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13125 entries, 0 to 13124
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    13125 non-null  object
 1   status  13125 non-null  object
dtypes: object(2)
memory usage: 205.2+ KB






## Предобработка данных

### Проверка корректности оформления столбцов

### Анализ пропущенных значений

### Проверка наличия явных и неявных дубикатов

## Исследовательский анализ данных

### Анализ количественных признаков

### Анализ категориальных признаков

### Проверка наличия дисбаланса в данных по целевой переменной

## Создание новых признаков

## Корреляционный анализ и финальный отбор признаков

## Разработка моделей МО

### Подготовка данных для моделей

### Обучение моделей в пайплайне

### Оценка результатов

## Анализ важности признаков и рекомендации по повышению шанса на успех стартапа

## Общий вывод